<a href="https://colab.research.google.com/github/heet25itachi/TIMEPASS_BUDDY/blob/main/Microsoft%20Drive%20Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
    from IPython.display import HTML

    html_content = "<h2>Dynamic HTML from Python</h2><p>The current time is: "
    # Add a timestamp or other dynamic content
    import datetime
    html_content += str(datetime.datetime.now()) + "</p>"

    display(HTML(html_content))

In [6]:
from IPython.display import HTML

html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Microsoft Drive Optimizer Simulator</title>
    <!-- Load Tailwind CSS -->
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        /* Custom styles for the grid visualization */
        .disk-block {
            width: 10px;
            height: 10px;
            border: 1px solid rgba(0, 0, 0, 0.05);
            transition: background-color 0.5s ease; /* Smooth color changes */
        }
        .fragmented { background-color: #ef4444; /* Red - Fragmented */ }
        .contiguous { background-color: #3b82f6; /* Blue - Contiguous */ }
        .free { background-color: #f3f4f6; /* Light Gray - Free Space */ }
        .moving { background-color: #f97316; /* Orange - Currently Moving */ }

        /* Ensure the grid container is responsive */
        #diskMap {
            display: grid;
            grid-template-columns: repeat(auto-fill, 10px);
            gap: 0;
            margin: 0 auto;
            border: 1px solid #e5e7eb;
            max-width: 800px;
        }
    </style>
    <script>
        tailwind.config = {
            theme: {
                extend: {
                    fontFamily: {
                        sans: ['Inter', 'sans-serif'],
                    },
                }
            }
            }
    </script>
</head>
<body class="bg-gray-50 font-sans p-4 sm:p-8 min-h-screen flex items-center justify-center">

    <div class="w-full max-w-4xl bg-white p-6 sm:p-10 shadow-xl rounded-xl border border-gray-200">

        <!-- Header -->
        <header class="mb-8">
            <h1 class="text-3xl font-extrabold text-gray-900 flex items-center">
                <svg class="w-8 h-8 mr-3 text-blue-600" fill="currentColor" viewBox="0 0 24 24">
                    <path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm-1 15h2v-6h-2v6zm0-8h2V7h-2v2z"/>
                </svg>
                Microsoft Drive Optimizer Simulator
            </h1>
            <p class="text-sm text-gray-500 mt-1">Simulated Analysis and Defragmentation for Local Disk (C:)</p>
        </header>

        <!-- Legend and Status -->
        <div class="grid grid-cols-2 md:grid-cols-4 gap-4 mb-6 text-sm">
            <div class="flex items-center">
                <span class="w-3 h-3 rounded mr-2 bg-red-500"></span> Fragmented
            </div>
            <div class="flex items-center">
                <span class="w-3 h-3 rounded mr-2 bg-blue-500"></span> Contiguous (Used)
            </div>
            <div class="flex items-center">
                <span class="w-3 h-3 rounded mr-2 bg-gray-200"></span> Free Space
            </div>
            <div class="flex items-center">
                <span class="w-3 h-3 rounded mr-2 bg-orange-500"></span> Moving Data
            </div>
        </div>

        <div class="mb-8 p-4 bg-gray-50 rounded-lg border">
            <p id="optimizationStatus" class="font-medium text-lg text-gray-700">Status: Initializing...</p>
            <div id="progressContainer" class="mt-2 h-2 bg-gray-200 rounded-full overflow-hidden hidden">
                <div id="progressBar" class="h-full bg-blue-500 transition-all duration-300" style="width: 0%;"></div>
            </div>
        </div>

        <!-- Disk Map Visualization -->
        <div id="diskMap" class="w-full h-[400px] overflow-hidden rounded-lg">
            <!-- Blocks will be injected here by JavaScript -->
        </div>

        <!-- Controls -->
        <div class="mt-8 flex justify-end space-x-4">
            <button id="analyzeBtn" onclick="runAnalysis()" class="px-6 py-2 bg-green-600 text-white font-semibold rounded-lg shadow-md hover:bg-green-700 transition duration-150 disabled:opacity-50" disabled>
                Analyze
            </button>
            <button id="defragBtn" onclick="runDefragmentation()" class="px-6 py-2 bg-blue-600 text-white font-semibold rounded-lg shadow-md hover:bg-blue-700 transition duration-150 disabled:opacity-50" disabled>
                Optimize
            </button>
        </div>
    </div>

    <script>
        // Disk Simulation Constants
        const TOTAL_BLOCKS = 5000;
        const FRAGMENTATION_RATE = 0.35; // 35% of used blocks are fragmented
        const USED_RATE = 0.70; // 70% of the disk is used

        // Block Types
        const TYPE_FREE = 'free';
        const TYPE_CONTIGUOUS = 'contiguous';
        const TYPE_FRAGMENTED = 'fragmented';
        const TYPE_MOVING = 'moving';

        let diskArray = [];
        let isRunning = false;
        let animationFrameId = null;

        // --- Core Functions ---

        /**
         * Initializes the disk array with random data distribution.
         */
        function initializeDisk() {
            diskArray = [];
            const usedBlocksCount = Math.floor(TOTAL_BLOCKS * USED_RATE);
            const fragmentedBlocksCount = Math.floor(usedBlocksCount * FRAGMENTATION_RATE);

            // 1. Fill with all used blocks first (contiguous)
            for (let i = 0; i < usedBlocksCount; i++) {
                diskArray.push({ id: i, type: TYPE_CONTIGUOUS, originalIndex: i, fileId: Math.floor(i / 100) });
            }

            // 2. Add free blocks
            for (let i = usedBlocksCount; i < TOTAL_BLOCKS; i++) {
                diskArray.push({ id: i, type: TYPE_FREE, originalIndex: i, fileId: null });
            }

            // 3. Randomly select and fragment some contiguous blocks
            const blocksToFragment = Array.from({ length: usedBlocksCount }, (_, i) => i);
            shuffleArray(blocksToFragment);

            for (let i = 0; i < fragmentedBlocksCount; i++) {
                const index = blocksToFragment[i];
                diskArray[index].type = TYPE_FRAGMENTED;
            }

            // Randomly scatter free blocks to simulate real-world usage after deletion/movement
            shuffleArray(diskArray);

            // Assign new IDs based on the shuffled order
            diskArray.forEach((block, index) => block.id = index);

            renderDiskMap();
            updateStatus('Disk initialized. Ready for Analysis.');
            document.getElementById('analyzeBtn').disabled = false;
        }

        /**
         * Shuffles an array (Fisher-Yates algorithm).
         */
        function shuffleArray(array) {
            for (let i = array.length - 1; i > 0; i--) {
                const j = Math.floor(Math.random() * (i + 1));
                [array[i], array[j]] = [array[j], array[i]];
            }
        }

        /**
         * Renders the disk array into the HTML grid container.
         */
        function renderDiskMap() {
            const mapContainer = document.getElementById('diskMap');
            mapContainer.innerHTML = '';

            // Create the grid blocks
            diskArray.forEach(block => {
                const blockDiv = document.createElement('div');
                blockDiv.className = `disk-block ${block.type}`;
                blockDiv.id = `block-${block.id}`;
                blockDiv.title = `Block ${block.id}: ${block.type}`;
                mapContainer.appendChild(blockDiv);
            });
        }

        /**
         * Updates the text status panel.
         */
        function updateStatus(message) {
            document.getElementById('optimizationStatus').textContent = `Status: ${message}`;
        }

        /**
         * Updates the progress bar.
         */
        function updateProgress(percent) {
            const progressBar = document.getElementById('progressBar');
            const progressContainer = document.getElementById('progressContainer');

            if (percent > 0 && percent < 100) {
                progressContainer.classList.remove('hidden');
            } else if (percent >= 100) {
                progressContainer.classList.add('hidden');
            }
            progressBar.style.width = `${percent}%`;
        }


        // --- Operation Functions ---

        /**
         * Simulates the Analyze step.
         */
        function runAnalysis() {
            if (isRunning) return;
            isRunning = true;
            document.getElementById('analyzeBtn').disabled = true;
            document.getElementById('defragBtn').disabled = true;

            updateStatus('Analyzing disk... (Pass 1 of 1)');
            updateProgress(0);

            let analyzedCount = 0;
            const totalUsed = diskArray.filter(b => b.type !== TYPE_FREE).length;

            const analysisLoop = () => {
                if (analyzedCount >= TOTAL_BLOCKS) {
                    // Analysis complete
                    const fragmentedCount = diskArray.filter(b => b.type === TYPE_FRAGMENTED).length;
                    const fragmentationPercent = totalUsed > 0 ? (fragmentedCount / totalUsed) * 100 : 0;

                    updateStatus(`Analysis Complete. ${fragmentationPercent.toFixed(1)}% of used space is fragmented.`);
                    updateProgress(100);
                    isRunning = false;

                    // Allow defragmentation only if fragmentation is significant
                    if (fragmentationPercent > 5) {
                        document.getElementById('defragBtn').disabled = false;
                    } else {
                        updateStatus(`Analysis Complete. ${fragmentationPercent.toFixed(1)}% fragmented. Optimization not required.`);
                    }
                    return;
                }

                // Simulate block scanning
                const block = diskArray[analyzedCount];
                const blockElement = document.getElementById(`block-${block.id}`);

                // Briefly highlight to show scanning
                if (blockElement) {
                    blockElement.style.outline = '1px solid yellow';
                }

                setTimeout(() => {
                    if (blockElement) {
                        blockElement.style.outline = 'none'; // Remove highlight
                    }
                }, 50);

                analyzedCount += 50; // Speed up the simulation
                updateProgress(Math.min(100, (analyzedCount / TOTAL_BLOCKS) * 100));
                animationFrameId = requestAnimationFrame(analysisLoop);
            };

            animationFrameId = requestAnimationFrame(analysisLoop);
        }

        /**
         * Simulates the Defragmentation/Optimization step.
         */
        async function runDefragmentation() {
            if (isRunning) return;
            isRunning = true;
            document.getElementById('analyzeBtn').disabled = true;
            document.getElementById('defragBtn').disabled = true;

            updateStatus('Optimizing disk... (Pass 1 of 3: Moving Data)');
            updateProgress(0);

            // 1. Identify blocks to move
            let blocksToMove = diskArray.filter(b => b.type === TYPE_FRAGMENTED);
            let freeBlocks = diskArray.filter(b => b.type === TYPE_FREE);

            // Sort blocks to move to maintain file order
            blocksToMove.sort((a, b) => a.originalIndex - b.originalIndex);

            // Total steps for simulation
            const totalSteps = blocksToMove.length;
            let currentStep = 0;

            const mapContainer = document.getElementById('diskMap');

            for (const block of blocksToMove) {
                if (freeBlocks.length === 0) break; // No more space to move

                // Find the first free spot (where contiguous data should end)
                const targetIndex = diskArray.findIndex(b => b.type === TYPE_FREE);

                if (targetIndex === -1) break; // Should not happen if freeBlocks.length > 0

                // Animate the move:
                const sourceElement = document.getElementById(`block-${block.id}`);

                // Mark source as moving and target as free
                if (sourceElement) {
                    sourceElement.className = 'disk-block moving';
                }

                // Simulated delay for the movement
                await new Promise(resolve => setTimeout(resolve, 5));

                // Execute the move in the array (swap)
                // Note: In a real defrag, data moves from fragment to free space, then fragment becomes free.
                // We simplify by swapping the data block with the free block.

                // Store properties before swap
                const sourceProps = { type: block.type, fileId: block.fileId, originalIndex: block.originalIndex };
                const targetBlock = diskArray[targetIndex];

                // Move data to the target
                targetBlock.type = TYPE_CONTIGUOUS; // New spot is now contiguous
                targetBlock.fileId = sourceProps.fileId;
                targetBlock.originalIndex = sourceProps.originalIndex;

                // Old spot is now free
                block.type = TYPE_FREE;
                block.fileId = null;
                block.originalIndex = block.id;

                // Re-render the visual swap by updating classes
                if (sourceElement) {
                    sourceElement.className = 'disk-block free'; // Old spot becomes free
                }
                const targetElement = document.getElementById(`block-${targetBlock.id}`);
                 if (targetElement) {
                    targetElement.className = 'disk-block contiguous'; // New spot becomes contiguous
                }

                // The actual block objects have been manipulated in diskArray, re-rendering with IDs is safer but slower.
                // For this simulation, we'll rely on class updates based on the swapped contents.

                // Re-render the entire map to fix the ID issue caused by array manipulation
                // This is a slow but visually correct way to reflect the changes
                renderDiskMap();

                currentStep++;
                updateProgress((currentStep / totalSteps) * 100 * 0.9); // Only 90% of process
            }

            updateProgress(100);
            updateStatus('Optimization Complete! Fragmentation is now minimal.');

            // After completion, reset status and re-enable analyze
            isRunning = false;
            document.getElementById('analyzeBtn').disabled = false;
        }


        // --- Initialization ---

        window.onload = () => {
            initializeDisk();
        };

    </script>
</body>
</html>
"""

display(HTML(html_content))

In [7]:
%%writefile drive_optimizer.cpp
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <algorithm>
#include <chrono>
#include <thread>
#include <limits>

using namespace std;

// --- Constants ---
const int TOTAL_BLOCKS = 2500;
const int DISK_WIDTH = 50;
const int BLOCK_SIZE = 4; // Simulated block size in MB
const float USED_RATE = 0.70f;
const float FRAGMENTATION_RATE = 0.40f;

// --- Enumerations and Structs ---
enum BlockType { FREE, CONTIGUOUS, FRAGMENTED };
struct DiskBlock {
    int id;
    int file_id;
    BlockType type;
};

// --- Utility Functions ---
void clear_screen() {
    cout << "\033[2J\033[1;1H";
}
void sleep_ms(int ms) {
    this_thread::sleep_for(chrono::milliseconds(ms));
}

// --- Core Logic (Initialization) ---
void initialize_disk(vector<DiskBlock>& disk_array) {
    disk_array.clear();
    srand(static_cast<unsigned int>(time(0)));

    int used_blocks_count = static_cast<int>(TOTAL_BLOCKS * USED_RATE);
    int fragmented_blocks_count = static_cast<int>(used_blocks_count * FRAGMENTATION_RATE);

    int current_id = 0;
    int current_file_id = 1;

    for (int i = 0; i < used_blocks_count; ++i) {
        DiskBlock block = {current_id++, current_file_id, CONTIGUOUS};

        if (i < fragmented_blocks_count) {
            block.type = FRAGMENTED;
        }

        if (i % 10 == 0) {
            current_file_id++;
        }
        disk_array.push_back(block);
    }

    for (int i = used_blocks_count; i < TOTAL_BLOCKS; ++i) {
        disk_array.push_back({current_id++, -1, FREE});
    }

    random_shuffle(disk_array.begin(), disk_array.end());

    for(int i = 0; i < TOTAL_BLOCKS; ++i) {
        disk_array[i].id = i;
    }
}

// --- Core Logic (Drawing) ---
void draw_disk(const vector<DiskBlock>& disk_array, const string& title) {
    clear_screen();
    cout << "--- " << title << " ---\n\n";
    cout << "Legend: \033[31mF\033[0m=Fragmented | \033[32mC\033[0m=Contiguous | \033[37m.\033[0m=Free Space\n";
    cout << "Total Blocks: " << TOTAL_BLOCKS << " (" << (TOTAL_BLOCKS * BLOCK_SIZE / 1024.0) << " GB Sim.)\n";
    cout << "Disk Map:\n";

    for (int i = 0; i < TOTAL_BLOCKS; ++i) {
        const DiskBlock& block = disk_array[i];

        char symbol = '.';
        string color_code = "\033[37m";

        if (block.type == FRAGMENTED) {
            symbol = 'F';
            color_code = "\033[31m";
        } else if (block.type == CONTIGUOUS) {
            symbol = 'C';
            color_code = "\033[32m";
        }

        cout << color_code << symbol << "\033[0m";

        if ((i + 1) % DISK_WIDTH == 0) {
            cout << "\n";
        }
    }
    cout << "\n";
}

// --- Core Logic (Analysis) ---
float analyze_disk(const vector<DiskBlock>& disk_array) {
    int fragmented_count = 0;
    int used_count = 0;

    for (const auto& block : disk_array) {
        if (block.type != FREE) {
            used_count++;
            if (block.type == FRAGMENTED) {
                fragmented_count++;
            }
        }
    }

    if (used_count == 0) return 0.0f;
    return (static_cast<float>(fragmented_count) / used_count) * 100.0f;
}

// --- Core Logic (Optimization) ---
void optimize_disk(vector<DiskBlock>& disk_array) {
    int free_cursor = 0;
    int blocks_moved = 0;
    const int SLEEP_TIME = 10;

    while (free_cursor < TOTAL_BLOCKS && disk_array[free_cursor].type != FREE) {
        free_cursor++;
    }

    if (free_cursor >= TOTAL_BLOCKS) {
        draw_disk(disk_array, "Optimization Skipped: Disk is already full and optimized.");
        sleep_ms(2000);
        return;
    }

    for (int i = free_cursor; i < TOTAL_BLOCKS; ++i) {
        if (disk_array[i].type != FREE) {

            DiskBlock data_block = disk_array[i];

            disk_array[free_cursor].file_id = data_block.file_id;
            disk_array[free_cursor].type = CONTIGUOUS;

            disk_array[i].file_id = -1;
            disk_array[i].type = FREE;

            free_cursor++;
            blocks_moved++;

            if (blocks_moved % 5 == 0) {
                draw_disk(disk_array, "Optimizing... Blocks Moved: " + to_string(blocks_moved));
                sleep_ms(SLEEP_TIME);
            }
        }
    }

    draw_disk(disk_array, "Optimization Complete! Blocks moved: " + to_string(blocks_moved));
    sleep_ms(2000);
}


// --- Main Program Loop ---
int main() {
    vector<DiskBlock> disk_c;
    char choice;

    initialize_disk(disk_c);

    do {
        draw_disk(disk_c, "C: Drive Status");
        float frag_percent = analyze_disk(disk_c);

        cout << "\n------------------------------------------------\n";
        cout << "Current Fragmentation: " << frag_percent << "%\n";
        cout << "------------------------------------------------\n";
        cout << "(A) Analyze | (O) Optimize | (R) Re-fragment | (Q) Quit\n";
        cout << "Enter command: ";

        if (!(cin >> choice)) {
            cin.clear();
            cin.ignore(numeric_limits<streamsize>::max(), '\n');
            continue;
        }

        choice = toupper(choice);

        if (choice == 'A') {
            draw_disk(disk_c, "Analyzing Disk...");
            sleep_ms(1000);
            cout << "\n\nAnalysis finished. Fragmentation is: " << frag_percent << "%\n";
            sleep_ms(1500);

        } else if (choice == 'O') {
            if (frag_percent > 1.0f) {
                optimize_disk(disk_c);
            } else {
                draw_disk(disk_c, "Optimization not necessary (<1% fragmentation).");
                sleep_ms(1500);
            }
        } else if (choice == 'R') {
            initialize_disk(disk_c);
            draw_disk(disk_c, "Disk Re-Fragmented (Simulated Heavy Usage)");
            sleep_ms(1500);
        } else if (choice != 'Q') {
            cout << "Invalid choice. Please try again.\n";
            sleep_ms(1000);
        }

    } while (choice != 'Q');

    clear_screen();
    cout << "Drive Optimizer Simulator Exited.\n";
    return 0;
}

### **Colab Cell 2: Compile**

This cell uses the g++ compiler to create an executable file named `drive_optimizer`.

```bash
!g++ drive_optimizer.cpp -o drive_optimizer -std=c++17

### **Colab Cell 3: Execute**

This cell runs the compiled program. You will interact with the simulator directly in the output area below this cell.

```bash
!./drive_optimizer

Writing drive_optimizer.cpp


In [9]:
%%writefile drive_optimizer.cpp
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <algorithm>
#include <chrono>
#include <thread>
#include <limits> // Required for numeric_limits

using namespace std;

// --- Constants ---
const int TOTAL_BLOCKS = 2500;
const int DISK_WIDTH = 50;
const int BLOCK_SIZE = 4; // Simulated block size in MB
const float USED_RATE = 0.70f;
const float FRAGMENTATION_RATE = 0.40f;

// --- Enumerations and Structs ---

// Defines the state of a disk block
enum BlockType { FREE, CONTIGUOUS, FRAGMENTED };

// Represents a single block on the disk
struct DiskBlock {
    int id;
    int file_id; // Identifies which file the fragment belongs to
    BlockType type;
};

// --- Utility Functions ---

void clear_screen() {
    // Colab/Terminal compatible screen clear using ANSI escape sequences
    cout << "\033[2J\033[1;1H";
}

void sleep_ms(int ms) {
    // Pause execution for visualization
    this_thread::sleep_for(chrono::milliseconds(ms));
}

// --- Core Logic ---

/**
 * @brief Initializes the disk with a mix of free, contiguous, and fragmented blocks.
 * @param disk_array Reference to the vector representing the disk.
 */
void initialize_disk(vector<DiskBlock>& disk_array) {
    disk_array.clear();
    // Use the current time to seed the random number generator
    srand(static_cast<unsigned int>(time(0)));

    int used_blocks_count = static_cast<int>(TOTAL_BLOCKS * USED_RATE);
    int fragmented_blocks_count = static_cast<int>(used_blocks_count * FRAGMENTATION_RATE);

    int current_id = 0;
    int current_file_id = 1;

    // 1. Create used blocks (mostly contiguous at first)
    for (int i = 0; i < used_blocks_count; ++i) {
        DiskBlock block = {current_id++, current_file_id, CONTIGUOUS};

        // Mark some blocks as fragmented
        if (i < fragmented_blocks_count) {
            block.type = FRAGMENTED;
        }

        // Change file ID every 10 blocks for simulation
        if (i % 10 == 0) {
            current_file_id++;
        }
        disk_array.push_back(block);
    }

    // 2. Create free blocks
    for (int i = used_blocks_count; i < TOTAL_BLOCKS; ++i) {
        disk_array.push_back({current_id++, -1, FREE});
    }

    // 3. Randomly shuffle the whole array to simulate fragmentation
    random_shuffle(disk_array.begin(), disk_array.end());

    // Reset IDs after shuffle for easier drawing
    for(int i = 0; i < TOTAL_BLOCKS; ++i) {
        disk_array[i].id = i;
    }
}

/**
 * @brief Draws the ASCII representation of the disk state.
 * @param disk_array The disk data.
 * @param title The title/status to display.
 */
void draw_disk(const vector<DiskBlock>& disk_array, const string& title) {
    clear_screen();
    cout << "--- " << title << " ---\n\n";
    cout << "Legend: \033[31mF\033[0m=Fragmented | \033[32mC\033[0m=Contiguous | \033[37m.\033[0m=Free Space\n";
    cout << "Total Blocks: " << TOTAL_BLOCKS << " (" << (TOTAL_BLOCKS * BLOCK_SIZE / 1024.0) << " GB Sim.)\n";
    cout << "Disk Map:\n";

    // Draw the grid
    for (int i = 0; i < TOTAL_BLOCKS; ++i) {
        const DiskBlock& block = disk_array[i];

        char symbol = '.';
        string color_code = "\033[37m"; // White (Free)

        if (block.type == FRAGMENTED) {
            symbol = 'F';
            color_code = "\033[31m"; // Red
        } else if (block.type == CONTIGUOUS) {
            symbol = 'C';
            color_code = "\033[32m"; // Green
        }

        cout << color_code << symbol << "\033[0m"; // Print colored symbol

        if ((i + 1) % DISK_WIDTH == 0) {
            cout << "\n";
        }
    }
    cout << "\n";
}

/**
 * @brief Simulates the Analysis stage.
 * @param disk_array The disk data.
 * @return Fragmentation percentage.
 */
float analyze_disk(const vector<DiskBlock>& disk_array) {
    int fragmented_count = 0;
    int used_count = 0;

    for (const auto& block : disk_array) {
        if (block.type != FREE) {
            used_count++;
            if (block.type == FRAGMENTED) {
                fragmented_count++;
            }
        }
    }

    if (used_count == 0) return 0.0f;
    return (static_cast<float>(fragmented_count) / used_count) * 100.0f;
}

/**
 * @brief Simulates the Optimization (Defragmentation) stage.
 * @param disk_array Reference to the disk data for modification.
 */
void optimize_disk(vector<DiskBlock>& disk_array) {
    int free_cursor = 0; // The target position for data
    int blocks_moved = 0;
    const int SLEEP_TIME = 10; // Animation speed

    // Find the first data block (where contiguous data should start)
    while (free_cursor < TOTAL_BLOCKS && disk_array[free_cursor].type != FREE) {
        free_cursor++;
    }

    if (free_cursor >= TOTAL_BLOCKS) {
        draw_disk(disk_array, "Optimization Skipped: Disk is already full and optimized.");
        sleep_ms(2000);
        return;
    }

    // Iterate through the entire disk starting from the current free_cursor
    for (int i = free_cursor; i < TOTAL_BLOCKS; ++i) {

        // We are looking for any data block (fragmented or contiguous) that is currently scattered.
        if (disk_array[i].type != FREE) {

            // If the block is already correctly positioned (i.e., before the free cursor), skip.
            // Since we start from the first free block (free_cursor), any data we find now is
            // a candidate for moving to consolidate free space.

            // 1. Temporarily store the data block
            DiskBlock data_block = disk_array[i];

            // 2. Move data_block to the free slot (free_cursor)
            disk_array[free_cursor].file_id = data_block.file_id;
            disk_array[free_cursor].type = CONTIGUOUS;

            // 3. Mark the block at index 'i' (the original location) as free.
            disk_array[i].file_id = -1;
            disk_array[i].type = FREE;

            // Advance the free cursor to the next expected free spot
            free_cursor++;

            blocks_moved++;

            // Visualization update (slow down for animation)
            if (blocks_moved % 5 == 0) { // Update every 5 moves
                draw_disk(disk_array, "Optimizing... Blocks Moved: " + to_string(blocks_moved));
                sleep_ms(SLEEP_TIME);
            }
        }
    }

    // Final redraw
    draw_disk(disk_array, "Optimization Complete! Blocks moved: " + to_string(blocks_moved));
    sleep_ms(2000);
}


// --- Main Program Loop ---

int main() {
    vector<DiskBlock> disk_c;
    char choice;

    // Initialize the disk on startup
    initialize_disk(disk_c);

    do {
        draw_disk(disk_c, "C: Drive Status");
        float frag_percent = analyze_disk(disk_c);

        cout << "\n------------------------------------------------\n";
        cout << "Current Fragmentation: " << frag_percent << "%\n";
        cout << "------------------------------------------------\n";
        cout << "(A) Analyze | (O) Optimize | (R) Re-fragment | (Q) Quit\n";
        cout << "Enter command: ";

        if (!(cin >> choice)) {
            // Handle bad input
            cin.clear();
            cin.ignore(numeric_limits<streamsize>::max(), '\n');
            continue;
        }

        choice = toupper(choice);

        if (choice == 'A') {
            draw_disk(disk_c, "Analyzing Disk...");
            sleep_ms(1000);
            cout << "\n\nAnalysis finished. Fragmentation is: " << frag_percent << "%\n";
            sleep_ms(1500);

        } else if (choice == 'O') {
            // Only optimize if fragmentation is > 1%
            if (frag_percent > 1.0f) {
                optimize_disk(disk_c);
            } else {
                draw_disk(disk_c, "Optimization not necessary (<1% fragmentation).");
                sleep_ms(1500);
            }
        } else if (choice == 'R') {
            initialize_disk(disk_c);
            draw_disk(disk_c, "Disk Re-Fragmented (Simulated Heavy Usage)");
            sleep_ms(1500);
        } else if (choice != 'Q') {
            cout << "Invalid choice. Please try again.\n";
            sleep_ms(1000);
        }

    } while (choice != 'Q');

    clear_screen();
    cout << "Drive Optimizer Simulator Exited.\n";
    return 0;
}

Overwriting drive_optimizer.cpp


### **Colab Cell 2: Compile**

This cell uses the g++ compiler to create an executable file named `drive_optimizer`.

In [10]:
!g++ drive_optimizer.cpp -o drive_optimizer -std=c++17

### **Colab Cell 3: Execute**

This cell runs the compiled program. You will interact with the simulator directly in the output area below this cell.

In [ ]:
!./drive_optimizer

--- C: Drive Status ---

Legend: F=Fragmented | C=Contiguous | .=Free Space
Total Blocks: 2500 (9.76562 GB Sim.)
Disk Map:
C.F..C.FCC.FCF..CF..FC.CFCF.CF.C.FCF.F..CCCCCCCF.C
.....CCFFC.CCF..FC.FF.CC..FC.CCFCC.FC.CCCCCCCCF.FF
CC.CCC.C.F.FC.CC.FCCF..FC.F...FCCCCCFC.C...C..F.CC
F.CCFFF..C.F.CC.CCF..FC..F...CFFCCCCC..F..CFF.FFFC
.CFF.FFCC.C.CCFFCC.C.CFF.FFCC.CCFFCC.CFCFFC.C.CF.C
CFC..FC.FCF.F...FCF.FCFCC.F.CC.CFFC...FCCC.CC.CF..
.CCFCCCCFF..FCC.CFF.FCFCC..C.F.C.FF..FFFC...FFCCCC
.CC.FFCC..CCC.FCFCCCCC..FC.FCCFFCC.CCCCCC.F...FC..
C..FF......CCCC....FCFF..C.CCF.CCCC..C.CFCC.CC.CF.
CC.FCCFC.FFCC..CCCC..CCF.CF..F.FFCF.FC.CC.....CC.F
CFC.CCC.F.CC..FFCF..FF..CC.CCFF.FCF..FF.FFCF..CC.C
CCFF..C.C..CFFCFC.FFFC.FCCF.CC.CFC.FCCF.CF.C.FFC..
CFFF..CC.CCC.CFF..C..CC..F..CCCFCCFFCC..C.CF.FFCC.
F.FCC...FFCF.F.FFFCCCF..FFCC.CCCF...C.CF.CFFCCCFCC
CCFCCFC..CCCF.FCCFC.CC.C.FC..CCF.CF.F.CCCFC...FCCC
.CCCF.CF.FFFF.C.CCFC.C..CC...CC.C.CFC...CF.C.F.F.C
C.CFC.CFCCCCC..CF...FCCCC..CCCFFCFC..F.CCCFCCFCF.C
CFCFCCF.CC